In [2]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.fasttext import load_facebook_vectors

In [ ]:
def find_matrix(lang, step=5000):
    
    print("LANG", lang)
    # not aligned
    try:
        src = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.bin") 
        print("Loaded fastText vectors")
    except:
        print("Going to 'vec'")
        src = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.vec")
    dst = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/aligned/wiki.{lang}.align.vec") # aligned
    
    if src.index_to_key != dst.index_to_key:
        print("src and dst vocabularies differ. ")
        print("src", len(src))
        print("dst", len(dst))
        print("in src, not in dst", set(src.index_to_key) - set(dst.index_to_key))
        print("in dst, not in src", set(dst.index_to_key) - set(src.index_to_key))
    
    vocab = list(set(src.index_to_key) & set(dst.index_to_key))
        
    Y = dst[vocab]
    X = src[vocab]

    W_ = np.linalg.pinv(X) @ Y

    prod = (X @ W_)
    prod = prod / np.linalg.norm(prod, axis=1).reshape(-1,1)

    error_couples = []
    right_values = []

    for i in range(0, len(prod), step):
            M = (prod[i:i+step] @ Y.T)
            v = M.argmax(axis=1)

            # sum of the diagonal
            right_values.append(np.diagonal(M[:,v]))
            for j in range(len(v)):
                if v[j] != i+j: # check that the most vector is the word itself
                    print("words do not match", i+j, v[j], M[j,v[j]])
                    print("instead the right word should be ", M[j,j])
                    error_couples.append((i+j, v[j], M[j,v[j]], M[j,j]))

                if M[j,v[j]] < .98:
                    print("small similarity" , i+j, v[j], M[j,v[j]])
    
    return src, dst, X, Y, W_, right_values.mean(), right_values.std(), error_couples

In [ ]:
src, dst, X, Y, W_, mean, std, error_couples = find_matrix("it")

# save the values 
import pickle
with open("it.pkl", "wb") as f:
    pickle.dump((src, dst, X, Y, W_, mean, std, error_couples), f)


In [ ]:
src, dst, X, Y, W_, mean, std, error_couples = find_matrix("it")

with open("it.pkl", "wb") as f:
    pickle.dump((src, dst, X, Y, W_, mean, std, error_couples), f)

In [ ]:
import pickle 

with open("res/it.pkl", "rb") as f:
    X, Y, W_, mean, std, error_couples = pickle.load(f)

In [ ]:
X.shape, Y.shape, W_.shape, mean, std

In [ ]:
error_couples

In [ ]:
lang = "it"

try:
    src = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.bin") 
    print("Loaded fastText vectors")
except:
    print("Going to 'vec'")
    src = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.vec")
dst = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/aligned/wiki.{lang}.align.vec") # aligned

In [ ]:
print(src.vectors_ngrams)

src.vectors.shape, src.vectors_ngrams.shape

In [ ]:
src.index_to_key[1000], src.buckets_word[1000]

In [ ]:
list = [] 
for i, n in enumerate(src.buckets_word): 
    # if in the list there is at least one index in src.buckets_word[1000], take the index 
    if len(set(n) & set(src.buckets_word[1000])) == 0 and i > 1000:
        list.append(i)
    if len(list) > 10:
        break

In [ ]:
for i in list:
    print(src.index_to_key[i])

In [ ]:
from copy import deepcopy

test = deepcopy(src.vectors_ngrams)
print(test.shape)

# change the vectors of the ngrams
for i in range(10):
    src.vectors_ngrams[i] = np.random.rand(300)

In [ ]:
test[0] == src.vectors_ngrams[0]

# test 19/01

In [4]:
lang = "it"

print("LANG", lang)
# not aligned
try:
    src1 = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.bin") 
    print("Loaded fastText vectors")
except:
    print("Going to 'vec'")
    src1 = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.vec")
dst1 = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/aligned/wiki.{lang}.align.vec") # aligned

if src1.index_to_key != dst1.index_to_key:
    print("src and dst1 vocabularies differ. ")
    print("src", len(src1))
    print("dst1", len(dst1))
    print("in src, not in dst1", set(src1.index_to_key) - set(dst1.index_to_key))
    print("in dst1, not in src1", set(dst1.index_to_key) - set(src1.index_to_key))

vocab = list(set(src1.index_to_key) & set(dst1.index_to_key))
    
Y1 = dst1[vocab]
X1 = src1[vocab]

LANG it
Loaded fastText vectors


In [49]:
import pickle 

with open("res/it.pkl", "rb") as f:
    src, dst, X, Y, W_, mean, std, error_couples = pickle.load(f)

In [35]:
(dst[1] == dst1[1]).all()

True

In [36]:
(X == X1).all()

False

In [38]:
X2 = src[vocab]
X3 = src1[vocab]
(X2 == X1).all(), (X3 == X1).all(), (X2 == X3).all(), (X2 == X).all(), (X3 == X).all()

(True, True, True, False, False)

In [42]:
X[0][2], X1[0][2]

(0.1637645, 0.14197187)

In [34]:
vocab = list(set(src1.index_to_key) & set(dst1.index_to_key))
vocab1 = list(set(src.index_to_key) & set(dst.index_to_key))

vocab[0], vocab1[0]

('chiedessero', 'chiedessero')

In [57]:
prod = (X @ W_)
prod_norm = prod / np.linalg.norm(prod, axis=1).reshape(-1,1)

In [50]:
v1 = []
v2 = []
for elem in error_couples:
    v1.append(elem[0])
    v2.append(elem[1])

In [60]:
for i in v1: 
    print(i, np.linalg.norm(prod[i]), np.linalg.norm(prod_norm[i]))

39181 2.2302065 0.99999994
58454 2.7192235 0.99999994
65725 2.8147001 1.0
140544 2.7426577 1.0
167916 2.6931403 0.99999994
199118 1.256816 1.0
264206 1.3493338 0.99999994
266438 2.4148433 1.0
352711 2.2067237 1.0
362468 2.8288927 0.99999994
435193 1.8902451 1.0
447450 2.1797674 1.0
460780 2.3000891 0.99999994
475714 2.8418713 1.0
581460 2.7993863 1.0
607079 1.9603934 0.99999994
627924 2.4457936 1.0
703267 2.3468404 1.0
716248 2.2692192 1.0
750082 2.0722136 0.99999994
755119 2.8751884 0.9999999
757323 2.7821946 1.0
764427 1.3468803 0.99999994
778784 1.2880855 1.0
790721 2.0233724 1.0
814669 2.3137376 1.0
837977 1.3491669 0.99999994
851996 1.3649734 0.99999994
866277 2.1138105 1.0


In [61]:
for i in v2: 
    print(i, np.linalg.norm(prod[i]), np.linalg.norm(prod_norm[i]))

703267 2.3468404 1.0
412293 2.7636247 0.99999994
475714 2.8418713 1.0
412293 2.7636247 0.99999994
140544 2.7426577 1.0
528129 1.2711023 1.0
272297 1.3820925 0.99999994
736928 2.4621701 1.0
703267 2.3468404 1.0
475714 2.8418713 1.0
607079 1.9603934 0.99999994
460780 2.3000891 0.99999994
627924 2.4457936 1.0
514556 2.8651068 1.0
475714 2.8418713 1.0
790721 2.0233724 1.0
736928 2.4621701 1.0
736928 2.4621701 1.0
627924 2.4457936 1.0
447450 2.1797674 1.0
502644 2.9568188 1.0
65725 2.8147001 1.0
475953 1.3878704 0.99999994
602250 1.3073055 0.99999994
866277 2.1138105 1.0
736928 2.4621701 1.0
446447 1.3667009 0.99999994
412384 1.4147501 1.0
352711 2.2067237 1.0


In [64]:
for i, j in zip(v1, v2): 
    print(dst.index_to_key[i], dst.index_to_key[j])

energica polylithionite
imax glystra
producing brinkhues
cassettonato glystra
spaghetto cassettonato
nabla himeshima
eucaliptus haldir
lecourt manocchi
chrebet polylithionite
dribblatore brinkhues
benedikte curatuli
zayton carimi
carimi σαν
brinkhues einn
thickets brinkhues
curatuli notoryctidae
σαν manocchi
polylithionite manocchi
└──>roberto σαν
ghdimes zayton
dissberger lashkargah
nfor producing
benici chūshirō
sučkoŭ tolina
notoryctidae qodirov
mentenere manocchi
nanoindentatore transmutations
piarista e^\,dx
qodirov chrebet
